<a href="https://colab.research.google.com/github/pahwa1902/MachineLearningProjects/blob/master/Lead_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [42]:
import pandas as pd
import seaborn as sns
import numpy as np

In [104]:
train_data=pd.read_csv('/content/drive/MyDrive/Analytics_vidhya_hackathon/train.csv')
test_data=pd.read_csv('/content/drive/MyDrive/Analytics_vidhya_hackathon/test.csv')

In [20]:
train_data.groupby(['Response']).size()

Response
0    38673
1    12209
dtype: int64

In [80]:
train_data.index

RangeIndex(start=0, stop=50882, step=1)

In [56]:
train_data[train_data['Health Indicator'].isnull()]

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
2,3,C5,3732,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,1
6,7,C3,679,Owned,Individual,28,28,No,NaN,NaN,NaN,17,10640.0,0
9,10,C1,530,Owned,Joint,59,26,Yes,NaN,7.0,4.0,18,21100.8,1
12,13,C7,3453,Owned,Individual,66,66,No,NaN,1.0,2.0,20,17192.0,1
19,20,C20,973,Owned,Individual,27,27,No,NaN,NaN,NaN,4,8050.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50859,50860,C1,217,Owned,Individual,70,70,No,NaN,6.0,3.0,20,19448.0,0
50865,50866,C21,4915,Owned,Individual,74,74,No,NaN,NaN,NaN,14,19944.0,0
50869,50870,C11,1770,Rented,Individual,45,45,No,NaN,1.0,1.0,20,10944.0,0
50871,50872,C10,224,Rented,Individual,21,21,No,NaN,1.0,1.0,13,11840.0,0


In [109]:
a=train_data[train_data['Response']==1].groupby(['Holding_Policy_Duration']).Response.agg(['count'])
b=train_data[train_data['Response']==0].groupby(['Holding_Policy_Duration']).Response.agg(['count'])


In [117]:
train_data['City_Code'].nunique()

36

In [77]:
b.reset_index(inplace=True)

In [ ]:
def duration(x,dic,result):
  if(x in dic):
    return dic[x]
  else:
    return result

In [ ]:
def transform(data):
  accomodation=pd.get_dummies(data['Accomodation_Type'])
  data['accomodation']=accomodation['Owned']

  data.drop(['Accomodation_Type'],axis=1,inplace=True)

  insurance=pd.get_dummies(data['Reco_Insurance_Type'])
  data['insurance']=insurance['Individual']
  data.drop(['Reco_Insurance_Type'],axis=1,inplace=True)

  spouse=pd.get_dummies(data['Is_Spouse'])
  data['spouse']=spouse['No']
  data.drop(['Is_Spouse'],axis=1,inplace=True)

  data['Reco_Policy_Premium']=np.sqrt(data['Reco_Policy_Premium'])
  health_index=data[data['Health Indicator'].isnull()].index

  health=pd.get_dummies(data['Health Indicator'])
  data=pd.concat([data,health],axis=1)
  data['health_null']=[1 for i in range(len(data)) if i in health_index else 0]
  data.drop(['Health Indicator'],axis=1,inplace=True)

  policy_index=data[data['Holding_Policy_Type'].isnull()].index
  policy=pd.get_dummies(data['Holding_Policy_Type'])
  policy=pd.concat([data,policy],axis=1)
  data['policy_null']=[1 for i in range(len(data)) if i in policy_index else 0]
  data.drop(['Holding_Policy_Type'],axis=1,inplace=True)

  data['average_age']=(data['Upper_Age']+data['Lower_Age'])//2
  data['average_age']=pd.qcut(data['average_age'],q=10)
  age=pd.get_dummies(data['average_age'])
  data=pd.concat([data,age],axis=1)
  data.drop(['Upper_Age','Lower_Age'],axis=1,inplace=True)
  
  a=data[data['Response']==1].groupby(['Holding_Policy_Duration']).Response.agg(['count'])
  b=data[data['Response']==0].groupby(['Holding_Policy_Duration']).Response.agg(['count'])

  dic=dict(b/a)
  duration_index=data[data['Holding_Policy_Duration'].isnull()].index
  positive=0
  negative=0
  for i in duration_index:
    if(data['Response']==1):
      positive+=1
    else:
      negative+=1
  data['Holding_Policy_Duration']=data['Holding_Policy_Duration'].apply(lambda x:duration(x,dic,negative/positive))

  city_1=data[data['Response']==1].groupby(['City_Code']).Response.agg(['count'])
  city_0=data[data['Response']==0].groupby(['City_Code']).Response.agg(['count'])

  dic1=dict(city_0/city_1)
  data['City_Code']=data['City_Code'].apply(lambda x:dic1[x])

  region=pd.qcut(data['Region_Code'],q=15)
  reqion=pd.get_dummies(region)
  data=pd.concat([data,region],axis=1)
  data.drop(['Region_Code','ID'],axis=1,inplace=True)
  
  return data, dic1, dic